# ETL?
## Extract
원시 데이터를 소스 위치에서 스테이징 영역으로 복사하거나 내보냄

다양한 소스에서 데이터를 추출할 수 있고, 이는 정형 또는 비정형

- 소스의 유형의 예시
    - SQL 또는 NoSQL 서버
    - CRM 및 ERP 시스템
    - 플랫 파일
    - 이메일
    - 웹페이지

## Transform
스테이징 영역에서 원시 데이터를 처리함

의도하는 분석 사용 사례에 맞게 변환 및 통합

다음과 같은 작업을 포함할 수 있음
- 데이터를 필터링, 정제, 중복 제거, 유효성 검사 및 인증
- 원시 데이터를 기반으로 계산, 번역 또는 요약을 수행합니다. 여기에는 일관성을 위해 행 및 열 머리글 변경, 통화 또는 기타 측정 단위 변환, 텍스트 문자열 편집 등이 포함될 수 있음
- 데이터 품질 및 규정 준수를 보장하기 위해 감사를 수행
- 업계 또는 정부 규제 기관에서 관리하는 데이터를 제거, 암호화 또는 보호
- 대상 데이터 웨어하우스의 스키마와 일치하도록 데이터를 테이블 또는 결합된 테이블로 포맷

## Load
변환된 데이터를 스테이징 영역에서 대상 데이터 웨어하우스로 이동
일반적으로 모든 데이터를 처음 로드한 다음 증분 데이터 변경 사항을 주기적으로 로드
드물게는 웨어하우스에서 데이터를 삭제하고 교체하는 전체 새로 고침을 수행하는 작업이 포함
일반적으로 ETL은 소스 시스템과 데이터 웨어하우스의 트래픽이 가장 적은 근무 외 시간에 이루어짐

In [12]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np


In [13]:
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
gdp_page = requests.get(url)
soup = bs(gdp_page.text, "html.parser")

In [14]:
gdps = []

# get table elements
trs = soup.select('table.wikitable > tbody > tr')

for tr in trs:
    td = tr.select('td')
    if td:
        country = td[0].text.strip()
        gdp = td[1].text.strip().replace(',','')
        if gdp == '—': # WARNING: this is not dash symbol
            gdp = '0' # 0 means NULL
        gdps.append((country, int(gdp) / 1000)) # from million to billion

gdps

[('World', 109529.216),
 ('United States', 28781.083),
 ('China', 18532.633),
 ('Germany', 4591.1),
 ('Japan', 4110.452),
 ('India', 3937.011),
 ('United Kingdom', 3495.261),
 ('France', 3130.014),
 ('Brazil', 2331.391),
 ('Italy', 2328.028),
 ('Canada', 2242.182),
 ('Russia', 2056.844),
 ('Mexico', 2017.025),
 ('Australia', 1790.348),
 ('South Korea', 1760.947),
 ('Spain', 1647.114),
 ('Indonesia', 1475.69),
 ('Netherlands', 1142.513),
 ('Turkey', 1113.561),
 ('Saudi Arabia', 1106.015),
 ('Switzerland', 938.458),
 ('Poland', 844.623),
 ('Taiwan', 802.958),
 ('Belgium', 655.192),
 ('Sweden', 623.048),
 ('Argentina', 604.26),
 ('Ireland', 564.02),
 ('Thailand', 548.89),
 ('Austria', 540.887),
 ('Israel', 530.664),
 ('United Arab Emirates', 527.796),
 ('Norway', 526.951),
 ('Singapore', 525.228),
 ('Philippines', 471.516),
 ('Vietnam', 465.814),
 ('Iran', 464.181),
 ('Bangladesh', 455.162),
 ('Malaysia', 445.519),
 ('Denmark', 409.989),
 ('Hong Kong', 406.775),
 ('Colombia', 386.076),
 (

In [16]:
column_name = ('Country', 'GDP')
df = pd.DataFrame(gdps, columns=column_name)
df_only_countries = df[df['Country']!='World'].sort_values(by='GDP', ascending=False).replace(0, np.NaN)
df_only_countries

,Country,GDP
1,United States,28781.083
2,China,18532.633
3,Germany,4591.100
4,Japan,4110.452
5,India,3937.011
...,...,...
158,Monaco,NaN
157,Syria,NaN
156,New Caledonia,NaN
143,North Korea,NaN


In [17]:
# json 파일로 저장
df_only_countries.to_json('Countries_by_GDP.json', orient='records', indent=4)